In [12]:
import import_ipynb
import numpy as np
import glob
import matplotlib.pyplot as plt
import datetime

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

## Model parameters

In [13]:
batch_size = 32
class_names = ['FaceOnly', 'NoFace']

## Model callbacks

In [14]:
tb_log_dir = "../log/tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
cp_filepath = '../log/cps/{model.name}_cp_{epoch}'

callbacks = [
    keras.callbacks.ModelCheckpoint(                        
                        filepath=cp_filepath, 
                        verbose=1, 
                        save_weights_only=False,
                        save_freq=5*batch_size), # Checkpoint is saved every five epochs
                        tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1)
]

## Data parameters

In [15]:
image_height = 224
image_width = 224

## Creating the training dataset

In [16]:
face_training_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    labels='inferred',
    label_mode='categorical',
    class_names=class_names,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

# normalization_layer = keras.layers.Rescaling(1./255)
# face_training_ds = face_training_ds.map(lambda x, y: (normalization_layer(x), y))
face_training_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 28 files belonging to 2 classes.
Using 23 files for training.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 2), dtype=tf.float32, name=None))>

## Creating the validation dataset

In [17]:
validation_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    labels='inferred',
    label_mode='categorical',
    class_names=class_names,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)
# validation_ds = validation_ds.map(lambda x, y: (normalization_layer(x), y))

Found 28 files belonging to 2 classes.
Using 5 files for validation.


## Creating the test dataset

In [18]:
test_ds = []

## Load the base model

In [19]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
base_model = ResNet50(
                include_top=False,
                weights='imagenet',  # Load weights pre-trained on ImageNet.
                input_shape=(image_height, image_width, 3)
                )
base_model.trainable = False

## Creating a new model

In [20]:
inputs = keras.Input(shape=(image_height, image_width, 3))

# Running base model in inference mode
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# Fully connected layer
x = keras.layers.Dense(1024, activation='relu')(x)

# Final layer for binary classification
outputs = keras.layers.Dense(2, activation='softmax')(x)

model = keras.Model(inputs, outputs)

## Training the new model with new data

In [21]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = model.fit(
            face_training_ds, 
            epochs=10, 
            callbacks=callbacks, 
            validation_data=validation_ds
        )

Epoch 1/10
1/1 [==============================] - 23s 23s/step - loss: 0.6801 - accuracy: 0.5217 - val_loss: 1.9147 - val_accuracy: 0.6000
Epoch 2/10
1/1 [==============================] - 10s 10s/step - loss: 2.7965 - accuracy: 0.4783 - val_loss: 0.5116 - val_accuracy: 0.8000
Epoch 3/10
1/1 [==============================] - 10s 10s/step - loss: 0.7768 - accuracy: 0.5217 - val_loss: 2.1280 - val_accuracy: 0.4000
Epoch 4/10
1/1 [==============================] - 10s 10s/step - loss: 1.7299 - accuracy: 0.5217 - val_loss: 2.5008 - val_accuracy: 0.4000
Epoch 5/10
1/1 [==============================] - 10s 10s/step - loss: 2.0229 - accuracy: 0.5217 - val_loss: 1.7759 - val_accuracy: 0.4000
Epoch 6/10
1/1 [==============================] - 10s 10s/step - loss: 1.4529 - accuracy: 0.5217 - val_loss: 0.7169 - val_accuracy: 0.4000
Epoch 7/10
1/1 [==============================] - 9s 9s/step - loss: 0.6779 - accuracy: 0.5217 - val_loss: 0.4779 - val_accuracy: 0.8000
Epoch 8/10
1/1 [=============

## View Results

## Predict new images

In [22]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../data/test/FaceTest/nonFace_10.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

#img_array = preprocess_input(x)

preds = model.predict(img_array)
#print('Predicted:', decode_predictions(preds, top=3)[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(preds[0])], 100 * np.max(preds[0]))
)

1/1 [==============================] - 2s 2s/step
This image most likely belongs to NoFace with a 99.99 percent confidence.
